In [1]:
import pickle
import gzip
from pathlib import Path

PATH = Path('/Users/seeni-2328/Documents/Seeni/Datasets/global-datasets/MNIST')
FILENAME = 'mnist.pkl.gz'
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [2]:
from matplotlib import pyplot
import numpy as np

pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)
print(x_train[0].shape)
# print(y_train.shape)
print(len(x_train), len(y_train), len(x_valid), len(y_valid))

(50000, 784)
(784,)
50000 50000 10000 10000


In [3]:
import torch
x_train, y_train, x_valid, y_valid = map(
        torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [4]:
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())
print(x_train.min(), x_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)
tensor(0.) tensor(0.9961)


In [39]:
from torch import nn

class Mnist_Logistic(nn.Module) :
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)
        
    def forward(self, xb) :
        return self.lin(xb)

In [7]:
# model = Mnist_Logistic()

In [13]:
import torch.nn.functional as F

loss_func = F.cross_entropy

In [14]:
bs = 64
xb = x_train[0:bs]
yb = y_train[0:bs]
print(loss_func(model(xb), yb))


tensor(2.2996, grad_fn=<NllLossBackward>)


In [34]:
from torch import optim

lr = 0.5  # learning rate
epochs = 2  # how many epochs to train for

def get_model() : # instead of creating a object for model, here we will initialise model with optimizer
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(), lr = lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

def fit() :
    for epoch in range(epochs) :
        for i in range((n-1)//bs+1) :
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)
            
            loss.backward()
            opt.step()
            opt.zero_grad()
#             with torch.no_grad() :
#                 for p in model.parameters() :
#                     p -= p.grad * lr
#                 model.zero_grad()
fit()

tensor(2.3156, grad_fn=<NllLossBackward>)


In [35]:
print(loss_func(model(xb), yb))

tensor(0.0814, grad_fn=<NllLossBackward>)


In [36]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(x_train, y_train)
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
#         xb = x_train[start_i:end_i]
#         yb = y_train[start_i:end_i]
        xb, yb = train_ds[start_i: end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))


tensor(0.0816, grad_fn=<NllLossBackward>)


In [38]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

model, opt = get_model()

for epoch in range(epochs):
#     for i in range((n - 1) // bs + 1):
#         start_i = i * bs
#         end_i = start_i + bs
    for xb, yb in train_dl : 
#         xb, yb = train_ds[start_i: end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0809, grad_fn=<NllLossBackward>)
